In [ ]:
FINAL_MODEL_PATH = os.path.join(SAVE_MODEL_PATH, "/kaggle/input/transform-eight/pytorch/default/1/scream_weight.pth") 
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
NUM_RUNS = 100  # Số lần lặp lại để tính trung bình
TEST_SIZES = [256, 512, 1024] 

def benchmark_speed(model, device, test_sizes, num_runs):
    """
    Đo thời gian suy luận trung bình (Inference Time) cho các kích thước ảnh khác nhau.
    """
    results = {}
    
    # Đặt mô hình vào chế độ đánh giá và chuyển sang GPU
    model.to(device).eval() 

    for size in test_sizes:
        print(f"\n--- Testing Size: {size}x{size} ---")
        
        # Tạo tensor ảnh giả (dummy input)
        dummy_input = torch.randn(1, 3, size, size).to(device)
        
        # WARM-UP RUNS (Đảm bảo GPU đã sẵn sàng và các kernel được khởi tạo)
        for _ in range(10):
            with torch.no_grad():
                _ = model(dummy_input)
        
        # ĐO LƯỜNG CHÍNH
        torch.cuda.synchronize() # Đồng bộ hóa GPU
        start_time = time.time()
        
        for _ in range(num_runs):
            with torch.no_grad():
                _ = model(dummy_input)
        
        torch.cuda.synchronize() # Chờ tất cả các phép tính GPU hoàn tất
        
        total_time = time.time() - start_time
        avg_time = total_time / num_runs
        fps = 1.0 / avg_time
        
        results[size] = {
            "avg_time_sec": avg_time,
            "fps": fps
        }
        print(f"  Avg Time: {avg_time:.4f}s | FPS: {fps:.2f}")

    return results
 
def run_benchmark():
    try:
        # Khởi tạo Mô hình (Chỉ cần cấu trúc, không cần trọng số VGG)
        model = TransformerNetModern() 
        
        # Tải trọng số cuối cùng của mô hình đã huấn luyện
        if os.path.exists(FINAL_MODEL_PATH):
            model.load_state_dict(torch.load(FINAL_MODEL_PATH, map_location=DEVICE))
            print(f"Loaded model weights from {FINAL_MODEL_PATH}")
        else:
            print(f"LỖI: Không tìm thấy file trọng số tại {FINAL_MODEL_PATH}. Vui lòng train mô hình trước.")
            return

        # Chạy Benchmark
        benchmark_data = benchmark_speed(model, DEVICE, TEST_SIZES, NUM_RUNS)

        # In Bảng Kết quả cho báo cáo
        print("\n\n=== BẢNG KẾT QUẢ HIỆU NĂNG CHO BÁO CÁO ===")
        print(f"{'Kích thước Ảnh':<20}{'Thời gian Xử lý (giây)':<25}{'Tốc độ (FPS)':<10}")
        print("-" * 55)
        for size, res in benchmark_data.items():
            print(f"{size} x {size:<13}{res['avg_time_sec']:.4f}{'s':<23}{res['fps']:.2f}")
        print("============================================")

    except NameError as e:
        print(f"LỖI KHỞI TẠO: {e}. Vui lòng đảm bảo class TransformerNetModern và VGG16 đã được định nghĩa.")

    except Exception as e:
            print(f"LỖI KHÔNG XÁC ĐỊNH: {e}")
 
run_benchmark()